In [1]:
from requests import get
from bs4 import BeautifulSoup
import time
import random
import html
import json
import datetime
import xml.etree.ElementTree

In [52]:
site_url = ""
url_filename = ""
url_list = []
for x in range(0,36):
    filename = 'sitemaps/sitemap-discussion-'+str(x)+'.xml/sitemap-discussion-'+str(x)+'.xml'
    e = xml.etree.ElementTree.parse(filename).getroot()
    for el in e:
        for url in el:
            group = url.text.replace(site_url,'')
            group = group[:group.find('/')]
            if group.lower() == 'anxiety':# or group.lower() == 'depression':
                url_list.append(url.text)
                #print(url.text)
                
print(len(url_list), 'urls extracted')
with open(url_filename, 'w', encoding='UTF-8') as txtstream:
    for url in url_list:
        txtstream.write(url+"\n")


37458 urls extracted


In [ ]:
print(len(url_list), 'urls extracted')
with open(url_filename, 'w', encoding='UTF-8') as txtstream:
    for url in url_list:
        txtstream.write(url+"\n")

In [ ]:
scrape_filename = ""
now = datetime.datetime.now()
print("Begin scraping", str(now))

urlnum = 0

with open(scrape_filename, 'w', encoding='UTF-8') as txtstream:
    for url in url_list:
        
        urlnum += 1
        
        try:
            response = get(url)

            html_soup = BeautifulSoup(response.text, 'html.parser')

            post_title = html_soup.find('div', class_ = 'group-breadcrumb__discussion-title')

            p_cont = html_soup.find('div', class_ = 'group-discussion-container__post')

            thread_pos = 0

            ptitle = post_title.h1.get_text()
            postedby = p_cont.find('span', class_ = 'newsfeed__posted-by').a.get_text()
            postedtime = p_cont.find('time', class_ = 'newsfeed__item-time')['datetime']
            pcontent = p_cont.find('div', class_ = 'posts__content').get_text().replace("\n"," ").replace("\r"," ").replace("\t"," ")
            totalcomments = p_cont.find('div', class_ = 'newsfeed__icon-count').get_text()

            txtstream.write(url+'|'+ptitle+'|'+str(thread_pos)+'|'+str(totalcomments)+'|'+postedby +'|'+ postedtime +'|'+ pcontent+'\n')
            print("Scraping post " + str(urlnum) + " - " + ptitle)

            thread_pos+=1    
            comment_containers = html_soup.find_all('div', class_ = 'comments__comment')
            for c_cont in comment_containers:

                postedby = c_cont.find('span', class_ = 'comments__name').a.get_text()
                postedtime = c_cont.find('time', class_ = 'newsfeed__itemtime')['datetime']
                pcontent = c_cont.find('div', class_ = 'comments__comment-text').get_text().replace("\n"," ").replace("\r"," ").replace("\t"," ")

                txtstream.write(url+'|'+ptitle+'|'+str(thread_pos)+'|'+str(totalcomments)+'|'+postedby +'|'+ postedtime +'|'+ pcontent+'\n')

                thread_pos+=1  
        except Exception as e:
             txtstream.write(url+'|'+str(e))
                             
        time.sleep(random.randint(1,4))

now = datetime.datetime.now()
print("End scraping", str(now))

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

df_posts = pd.read_csv(scrape_filename, sep="|", header=None, names = ["url", "title", "thread_pos", "comment_num", "posted_by", 
                                                            "posted_time", "content"], dtype={"url": str, "title": str, "thread_pos": int,"comment_num": int, "posted_by": str, 
                                                            "posted_time":str, "content":str}, parse_dates=["posted_time"])

df_posts.head(50)